In [1]:
using Pkg
Pkg.activate("/media/mat/HDD/AdaptiveTransportMap/")

 Activating environment at `/media/mat/HDD/AdaptiveTransportMap/Project.toml`


In [2]:
using Revise
using AdaptiveTransportMap
using LinearAlgebra
import LinearAlgebra: ldiv!, dot
using Test
using ForwardDiff
using SpecialFunctions
using BenchmarkTools
using TransportMap
using QuadGK
using Polynomials
using Distributions
using Random
using LoopVectorization
using Optim
using NLsolve
using MLDataUtils
using MLDataPattern
using Test
using SparseArrays
using QRupdate
using AdaptiveTransportMap: vander, transform!, evaluate, ncoeff, optimize, negative_log_likelihood!, derivative

# using Profile
# using ProfileView

### Test optimization with QR

In [3]:
Nx = 6
m = 15
Ne = 100

X = randn(Nx, Ne) .* randn(Nx, Ne)


L = LinearTransform(X)
transform!(L, X)

6×100 Array{Float64,2}:
  0.263081  -0.217364  -0.100711   …  0.452757   0.2903      0.0657519
  0.112594  -1.12226    0.0999967     0.233185   0.388683    0.134396
  0.788453   0.193321   1.80094       0.212999   0.394632   -0.0914674
 -0.755565   1.11549   -1.67948       0.96866    0.50064    -0.0308019
 -1.94085   -0.53971   -0.0983404     1.12427   -0.0056782  -1.60809
 -0.216443  -1.99926   -0.0608417  …  0.112603  -1.92905    -0.888565

In [19]:
@testset "Test optimization with QR basis and Hessian preconditioner without/with L2 penalty" begin

    Nx = 3
    Ne = 8
    m = 20

    idx = [0 0 0; 0 0 1; 0 1 0; 0 1 1; 0 1 2; 1 0 0]

    Nψ = 6
    coeff = [ 0.20649582065364197;
             -0.5150990160472986;
              2.630096893080717;
              1.13653076177397;
              0.6725837371023421;
             -1.3126095306624133]
    coeff0 = deepcopy(coeff)

    X =  Matrix([ 1.12488     0.0236348   -0.958426;
             -0.0493163   0.00323509  -0.276744;
              1.11409     0.976117     0.256577;
             -0.563545    0.179956    -0.418444;
              0.0780599   0.371091    -0.742342;
              1.77185    -0.175635     0.32151;
             -0.869045   -0.0570977   -1.06254;
             -0.165249   -2.70636      0.548725]')

    L = LinearTransform(X)
    transform!(L, X);

    # For α = 0.0
    C = MapComponent(m, Nx, idx, coeff; α = 0.0)

    S = Storage(C.I.f, X)
    S̃ = deepcopy(S)
    F = QRscaling(S̃)


    res = Optim.optimize(Optim.only_fg!(negative_log_likelihood(S, C, X)), coeff, Optim.LBFGS(; m = 20))

    # In QR space
    c̃oeff0 = F.U*coeff0

    mul!(S̃.ψoffψd0, S̃.ψoffψd0, F.Uinv)
    mul!(S̃.ψoffdψxd, S̃.ψoffdψxd, F.Uinv)

    r̃es = Optim.optimize(Optim.only_fg!(qrnegative_log_likelihood(F, S̃, C, X)), c̃oeff0, Optim.LBFGS(; m = 20))

    # With QR and Hessian approximation

    qrprecond = zeros(ncoeff(C), ncoeff(C))
    qrprecond!(qrprecond, c̃oeff0, F, S̃, C, X)

    r̃esprecond = Optim.optimize(Optim.only_fg!(qrnegative_log_likelihood(F, S̃, C, X)), c̃oeff0,
                           Optim.LBFGS(; m = 20, P = Preconditioner(qrprecond)))

    @test norm(res.minimizer - F.Uinv*r̃es.minimizer)<1e-5
    @test norm(res.minimizer - F.Uinv*r̃esprecond.minimizer)<1e-5
    @test norm(r̃es.minimizer - r̃esprecond.minimizer)<1e-5


    # For α = 1e-1
    C = MapComponent(m, Nx, idx, coeff; α = 0.1)

    S = Storage(C.I.f, X)
    S̃ = deepcopy(S)
    F = QRscaling(S̃)


    res = Optim.optimize(Optim.only_fg!(negative_log_likelihood(S, C, X)), coeff, Optim.LBFGS(; m = 20))

    # With QR
    c̃oeff0 = F.U*coeff0

    mul!(S̃.ψoffψd0, S̃.ψoffψd0, F.Uinv)
    mul!(S̃.ψoffdψxd, S̃.ψoffdψxd, F.Uinv)

    r̃es = Optim.optimize(Optim.only_fg!(qrnegative_log_likelihood(F, S̃, C, X)), c̃oeff0, Optim.LBFGS(; m = 20))

    @test norm(res.minimizer - F.Uinv*r̃es.minimizer)<1e-5

    # With QR and Hessian approximation

    qrprecond = zeros(ncoeff(C), ncoeff(C))
    qrprecond!(qrprecond, c̃oeff0, F, S̃, C, X)

    r̃esprecond = Optim.optimize(Optim.only_fg!(qrnegative_log_likelihood(F, S̃, C, X)), c̃oeff0,
                           Optim.LBFGS(; m = 20, P = Preconditioner(qrprecond)))

    @test norm(res.minimizer - F.Uinv*r̃esprecond.minimizer)<1e-5
    @test norm(r̃es.minimizer - r̃esprecond.minimizer)<1e-5
end

Test Summary:                                                                      | Pass  Total
Test optimization with QR basis and Hessian preconditioner without/with L2 penalty |    6      6


Test.DefaultTestSet("Test optimization with QR basis and Hessian preconditioner without/with L2 penalty", Any[], 6, false)

In [18]:
ncoeff(C)

1

### Integration

In [24]:
Nx = 100
Ny = 50
m = 20
Ne = 400
X = randn(Nx, Ne).*randn(Nx, Ne) #.+ 0.5*randn(Nx).*cos.(randn(Nx, Ne) .* randn(Nx, Ne))

# L = LinearTransform(X)
# transform!(L, X);
# C = MapComponent(m, Nx)
# @time C, _ = optimize(C, X, 10)

LoadError: AssertionError: Size of the array of multi-indices idx is wrong

In [22]:
M = HermiteMap(m, X);

In [9]:
@time optimize(M, X, 10; P = serial);

 11.061632 seconds (49.90 M allocations: 19.060 GiB, 7.20% gc time)


In [10]:
function timing()
    @btime begin 
        M = HermiteMap($m, $X)
        optimize($M, $X, $10; P = serial, start = Ny+1)
    end
end

timing (generic function with 1 method)

In [11]:
timing();

  14.597 s (42433511 allocations: 13.37 GiB)


In [29]:
timing();

  973.450 ms (1054425 allocations: 1.43 GiB)


In [14]:
methods(update_component)

# 1 method for generic function "update_component":
[1] update_component(C::MapComponent, X, reduced_margin::Array{Int64,2}, S::Storage) in AdaptiveTransportMap at /media/mat/HDD/AdaptiveTransportMap/src/mapcomponent/greedyfit.jl:249

In [30]:
function timing()
    @btime begin 
        M = HermiteMap($m, $X)
        optimize($M, $X, $10; P = thread)
    end
end

timing (generic function with 1 method)

In [88]:
timing();

  297.730 ms (1000725 allocations: 1.35 GiB)
